In [21]:
import yfinance as yf
import pandas as pd

from datetime import time, datetime, timedelta






In [22]:
ticker = 'GOOGL'


tickerdata = yf.Ticker(ticker)

tickerdata.info

{'address1': '1600 Amphitheatre Parkway',
 'city': 'Mountain View',
 'state': 'CA',
 'zip': '94043',
 'country': 'United States',
 'phone': '650 253 0000',
 'website': 'https://abc.xyz',
 'industry': 'Internet Content & Information',
 'industryKey': 'internet-content-information',
 'industryDisp': 'Internet Content & Information',
 'sector': 'Communication Services',
 'sectorKey': 'communication-services',
 'sectorDisp': 'Communication Services',
 'longBusinessSummary': 'Alphabet Inc. offers various products and platforms in the United States, Europe, the Middle East, Africa, the Asia-Pacific, Canada, and Latin America. It operates through Google Services, Google Cloud, and Other Bets segments. The Google Services segment provides products and services, including ads, Android, Chrome, devices, Gmail, Google Drive, Google Maps, Google Photos, Google Play, Search, and YouTube. It is also involved in the sale of apps and in-app purchases and digital content in the Google Play and YouTube;

In [23]:
# getting historical market data

tickerdf = tickerdata.history(period='1d', start='2024-1-1', end='2024-4-1')

print(tickerdf)

                                 Open        High         Low       Close  \
Date                                                                        
2024-01-02 00:00:00-05:00  138.550003  139.449997  136.479996  138.169998   
2024-01-03 00:00:00-05:00  137.250000  139.630005  137.080002  138.919998   
2024-01-04 00:00:00-05:00  138.419998  139.160004  136.350006  136.389999   
2024-01-05 00:00:00-05:00  136.750000  137.160004  135.149994  135.729996   
2024-01-08 00:00:00-05:00  136.289993  139.009995  136.259995  138.839996   
...                               ...         ...         ...         ...   
2024-03-22 00:00:00-04:00  149.119995  151.580002  148.979996  150.770004   
2024-03-25 00:00:00-04:00  149.940002  150.380005  147.820007  150.070007   
2024-03-26 00:00:00-04:00  150.220001  152.259995  149.979996  150.669998   
2024-03-27 00:00:00-04:00  151.179993  151.639999  148.899994  150.869995   
2024-03-28 00:00:00-04:00  150.850006  151.429993  150.169998  150.929993   

In [24]:
# last closing data

print(tickerdf['Close'].iloc[-1])





150.92999267578125


In [25]:
tickerdf.shape

(61, 7)

In [32]:
# checking current price

end_date = datetime.now()

end_date = end_date.strftime("%Y-%m-%d")

start_date = datetime.now() - timedelta(days=365)

start_date = start_date.strftime("%Y-%m-%d")

print(start_date)

print(end_date)




2023-04-26
2024-04-25
